In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import random as ran
from IPython.display import clear_output
from src import spotify, dp, utils, DB
import pandas as pd

db = DB()

In [3]:
genres_cat = ['k-pop', 'pop', 'hip-hop', 'ballad', 'jazz', "test"]

genres_dict = dict({
    "k-pop": "3dbLT62Cvs46Ju7a8gpr36",
    "pop": "3gzIK2dGSVvPBiohsTwEWZ",
    "hip-hop": "2FWquqPNxte8iqZ3ATQG0p",
    "ballad": "3UPjb91Fwm7u2tAm92Bk0p",
    "jazz": "7tIOwGFRTM08JdAeKOELGp",
    "test": "22skzmqfdWrjJylampe0kt"
})

In [4]:
mail_box_seed_list = list()

for _ in range(0, 10):
    genre_cnt = ran.randrange(1, 4)
    genre_cat = list()
    
    while len(genre_cat) != genre_cnt:
        ran_genres_cat = genres_cat[
            ran.randrange(0, len(genres_cat))
        ]
        
        if ran_genres_cat not in genre_cat:
            genre_cat.append(ran_genres_cat)

    mail_box_seed_list.append(genre_cat)
    
mail_box_seed_list

[['ballad'],
 ['k-pop'],
 ['test', 'ballad'],
 ['ballad', 'jazz'],
 ['ballad', 'test'],
 ['test'],
 ['hip-hop', 'pop', 'k-pop'],
 ['pop', 'ballad', 'k-pop'],
 ['ballad', 'test'],
 ['ballad']]

In [5]:
# 1. Get Token
token = spotify.get_token()

mail_boxes_list = list()
sel_track_list = list()
sel_tracks = pd.DataFrame()

for box_seed in mail_box_seed_list:
    sel_tracks = pd.DataFrame()
    
    for seed_genre in box_seed:
        seed_id = genres_dict[seed_genre]
        
        sel_tracks = sel_tracks.append(spotify.direct_sel(token, seed_id), ignore_index=True)
    
    sel_track_list.append(sel_tracks)
    mail_box_id = str(db.regist_mail_box(sel_tracks).inserted_id)
    
    mail_boxes_list.append(mail_box_id)

print(mail_boxes_list)
sel_track_list[0]

['6242bbaad8bbb8baa1d8baae', '6242bbabd8bbb8baa1d8baaf', '6242bbabd8bbb8baa1d8bab0', '6242bbabd8bbb8baa1d8bab1', '6242bbabd8bbb8baa1d8bab2', '6242bbabd8bbb8baa1d8bab3', '6242bbabd8bbb8baa1d8bab4', '6242bbacd8bbb8baa1d8bab5', '6242bbacd8bbb8baa1d8bab6', '6242bbacd8bbb8baa1d8bab7']


,id,name,artists,artists_name
0,3UPjb91Fwm7u2tAm92Bk0p,why break up?,02SENF0XVfxRTZrniEV2aF,Sin Ye Young


In [6]:
features_list = list()
genre_seed_list = list()
for sel_track in sel_track_list:
    genres = spotify.get_genres(token, sel_track)
    genre_seed_list.append(genres)
    
    features = spotify.get_features(token, sel_track)
    features_list.append(features)
    
    db.observe_seed_zone(features)
    
print(genre_seed_list[9])
features_list[9]

                  artists     genres
0  02SENF0XVfxRTZrniEV2aF  k-pop,pop


,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3UPjb91Fwm7u2tAm92Bk0p,0.554,0.526,7,-5.21,0.0394,0.808,0,0.074,0.23,129.845


In [7]:
reco_list = list()
reco_features_list = list()

for idx, sel_track in enumerate(sel_track_list):
    print(idx)
    features = features_list[idx]
    genres = genre_seed_list[idx]
    
    recos = spotify.get_recommend(sel_track, features, genres, token)
    reco_features = spotify.get_features(token, recos)
    
    reco_list.append(recos)
    reco_features_list.append(reco_features)

0
1
2
3
4
5
6
7
8
9


In [8]:
reco_list[0].head()

,id,name,artists,artists_name
0,46ydq5g3k17iLJs3qMDvO6,Hurts So Good,3AVfmawzu83sp94QW7CEGm,Astrid S
1,4TlN6gykUprYUyjaI8CeQD,Here To Stay - SKULL featuring Mighty Mystic,7qKWQnE3S7v8m3rLwMVdGg,SKULL
3,4so85rrFc5TCgT6rd2vweV,서툰 이별을 하려해,13PsdrWmNx5EcibArzkbcO,Yountoven
4,3ZlxwbbbmEqRZsOjl06Qii,Waiting for Spring,6pU8o91xAS0aWNjj06nQSU,Kassy
5,6EWWZTKKhFh7sTYsm65zfV,0310,6dhfy4ByARPJdPtMyrUYJK,Yerin Baek


In [9]:
reco_features_list[0].head()

,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,46ydq5g3k17iLJs3qMDvO6,0.675,0.609,7,-5.027,0.0586,0.084,0.000000,0.0957,0.378,120.039
1,4TlN6gykUprYUyjaI8CeQD,0.619,0.737,8,-5.052,0.0338,0.291,0.000000,0.1310,0.948,93.921
2,4so85rrFc5TCgT6rd2vweV,0.538,0.470,7,-7.111,0.0378,0.820,0.000000,0.2430,0.196,133.585
3,3ZlxwbbbmEqRZsOjl06Qii,0.501,0.533,7,-4.844,0.0329,0.846,0.000000,0.1230,0.196,139.619
4,6EWWZTKKhFh7sTYsm65zfV,0.548,0.459,7,-6.467,0.0304,0.703,0.000015,0.0989,0.525,135.975


In [10]:
norm_features_list = list()

for idx, features in enumerate(features_list):
    reco_features = reco_features_list[idx]
    norm_features = dp.make_norm(features, reco_features)
    norm_features_list.append(norm_features)
    
norm_features_list[0].head()

,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3UPjb91Fwm7u2tAm92Bk0p,0.480769,0.4752,0.636364,0.654057,0.031550,0.895221,0.0,0.101194,0.157079,0.556688
1,46ydq5g3k17iLJs3qMDvO6,0.692308,0.6080,0.636364,0.683516,0.075446,0.088203,0.0,0.169390,0.330829,0.469987
2,4TlN6gykUprYUyjaI8CeQD,0.594406,0.8128,0.727273,0.679491,0.018747,0.318939,0.0,0.280327,1.000000,0.239061
3,4so85rrFc5TCgT6rd2vweV,0.452797,0.3856,0.636364,0.348036,0.027892,0.908597,0.0,0.632307,0.117164,0.589756
4,3ZlxwbbbmEqRZsOjl06Qii,0.388112,0.4864,0.636364,0.712975,0.016690,0.937579,0.0,0.255185,0.117164,0.643107


In [11]:
kmeans_list = list()

for idx, norm_features in enumerate(norm_features_list):
    print("-- 라운드 {}".format(idx + 1))
    kmeans = utils.KMeans(
        datas=norm_features
    )
    kmeans.run(early_stop_cnt=3)
    clear_output(wait=True)
    
    kmeans_list.append(kmeans)

1 round : ECV 13.98%
2 round : ECV 45.74%
3 round : ECV 48.7%
4 round : ECV 49.86%
5 round : ECV 50.02%
6 round : ECV 50.32%
7 round : ECV 50.41%
8 round : ECV 50.41%
9 round : ECV 50.41%
10 round : ECV 50.41%
11 round : ECV 50.41%
[48.25329383 52.1233726  58.08457123 48.67110652 48.29540188 50.41298604]
Clustering End. ECV 58.08457122874713 data Select


In [12]:
filterings = list()
recos_list = list()

for idx, kmeans in enumerate(kmeans_list):
    sel_track = sel_track_list[idx]
    recos = reco_list[idx]
    
    _filtering_music_list = dp.music_filtering(sel_track, kmeans)
    filterings.append(_filtering_music_list)
    
    reco_musics = [_ in _filtering_music_list for _ in recos['id']]
    recos_list.append(recos[reco_musics])
    
recos_list[0].head()

,id,name,artists,artists_name
4,3ZlxwbbbmEqRZsOjl06Qii,Waiting for Spring,6pU8o91xAS0aWNjj06nQSU,Kassy
5,6EWWZTKKhFh7sTYsm65zfV,0310,6dhfy4ByARPJdPtMyrUYJK,Yerin Baek
9,0ovFZW9wzvA5SUosXSUB5Z,SULLAE,0jUn8CXobOt0IixyR72una,Rothy
10,741iHAqg8dbs8vaNnnMDve,Greeting,6pU8o91xAS0aWNjj06nQSU,Kassy
13,22CY3wlF8O6Z9QkOWYhaz5,눈 떠보니 이별이더라,7oFIkpNpLrTBgZW6w55W1J,4MEN


In [13]:
for idx,_mail in enumerate(recos_list):
    mail_box_id = mail_boxes_list[idx]
    
    res = db.regist_mail(mail_box_id, _mail)
    mail_id = res.inserted_id
    
    print("우체통({})에\n추천 플레이리스트 MuLetter({})가 등록되었습니다.".format(
        mail_box_id
        ,str(mail_id)))

우체통(6242bbaad8bbb8baa1d8baae)에
추천 플레이리스트 MuLetter(6242bbd9d8bbb8baa1d8bab8)가 등록되었습니다.
우체통(6242bbabd8bbb8baa1d8baaf)에
추천 플레이리스트 MuLetter(6242bbd9d8bbb8baa1d8bab9)가 등록되었습니다.
우체통(6242bbabd8bbb8baa1d8bab0)에
추천 플레이리스트 MuLetter(6242bbd9d8bbb8baa1d8baba)가 등록되었습니다.
우체통(6242bbabd8bbb8baa1d8bab1)에
추천 플레이리스트 MuLetter(6242bbd9d8bbb8baa1d8babb)가 등록되었습니다.
우체통(6242bbabd8bbb8baa1d8bab2)에
추천 플레이리스트 MuLetter(6242bbd9d8bbb8baa1d8babc)가 등록되었습니다.
우체통(6242bbabd8bbb8baa1d8bab3)에
추천 플레이리스트 MuLetter(6242bbd9d8bbb8baa1d8babd)가 등록되었습니다.
우체통(6242bbabd8bbb8baa1d8bab4)에
추천 플레이리스트 MuLetter(6242bbd9d8bbb8baa1d8babe)가 등록되었습니다.
우체통(6242bbacd8bbb8baa1d8bab5)에
추천 플레이리스트 MuLetter(6242bbd9d8bbb8baa1d8babf)가 등록되었습니다.
우체통(6242bbacd8bbb8baa1d8bab6)에
추천 플레이리스트 MuLetter(6242bbd9d8bbb8baa1d8bac0)가 등록되었습니다.
우체통(6242bbacd8bbb8baa1d8bab7)에
추천 플레이리스트 MuLetter(6242bbd9d8bbb8baa1d8bac1)가 등록되었습니다.


### reply

In [14]:
import math as mt

In [15]:
track_list = list()
reco_track_list = list()

for mail_box_id in mail_boxes_list:
    tracks = db.get_mail_box(mail_box_id)
    reco_tracks = db.get_mail(mail_box_id)

    track_list.append(tracks)
    reco_track_list.append(reco_tracks)
    
track_list[0].head()

,id,name,artists,artists_name
0,3UPjb91Fwm7u2tAm92Bk0p,why break up?,02SENF0XVfxRTZrniEV2aF,Sin Ye Young


In [16]:
sel_track_list = list()

for idx, reco_tracks in enumerate(reco_track_list):
    mail_box_id = mail_boxes_list[idx]
    n = mt.floor(len(reco_tracks) * 0.075)
    n = 2 if n <= 1 else n
    
    sel_tracks = reco_tracks.sample(n=n)
    db.new_mail_box_seed(mail_box_id, sel_tracks)

    sel_track_list.append(sel_tracks)

In [17]:
# 1. Get Token
token = spotify.get_token()

_sel_track_list = list()
features_list = list()
recos_list = list()
reco_features_list = list()

for idx, sel_tracks in enumerate(sel_track_list):
    print(idx)
    tracks = track_list[idx]
    _sel_tracks = pd.concat([tracks,sel_tracks], ignore_index=True)
    genres = spotify.get_genres(token, _sel_tracks)

    og_features = spotify.get_features(token, tracks)
    reply_features = spotify.get_features(token, sel_tracks)
    db.observe_seed_zone(reply_features)

    recos = spotify.get_recommend(sel_tracks, reply_features, genres, token,
                                 og=tracks)
    
    if str(type(recos)) == "<class 'requests.models.Response'>":
        print(recos.url)
        print(idx,recos.json())
        
    reco_features = spotify.get_features(token, recos)
    features = pd.concat([og_features, reply_features], ignore_index=True)
    
    features_list.append(features)
    recos_list.append(recos)
    reco_features_list.append(reco_features)
    _sel_track_list.append(_sel_tracks)

0
1
2
3
4
5
6
7
8
9


In [18]:
norm_features_list = list()

for idx, reco_features in enumerate(reco_features_list):
    features = features_list[idx]
    norm_features = dp.make_norm(features, reco_features)

    norm_features_list.append(norm_features)

In [19]:
filterings = list()
kmeans_list = list()

for idx, sel_tracks in enumerate(sel_track_list):
    print("{}번 박스".format(idx))
    norm_features = norm_features_list[idx]
    
    _1, _2 = utils.reco_KMeans(norm_features, sel_tracks)
    
    filterings.append(_1)
    kmeans_list.append(_2)

1 round : ECV 28.98%
2 round : ECV 54.04%
3 round : ECV 58.43%
4 round : ECV 60.1%
5 round : ECV 60.92%
6 round : ECV 61.92%
7 round : ECV 62.97%
8 round : ECV 63.44%
9 round : ECV 63.67%
10 round : ECV 63.98%
11 round : ECV 64.11%
12 round : ECV 64.11%
13 round : ECV 64.11%
14 round : ECV 64.11%
15 round : ECV 64.11%
16 round : ECV 64.11%
17 round : ECV 64.11%
[63.28547608 62.99182339 61.45963906 62.12562415 61.92970083 64.10574011]
Clustering End. ECV 64.10574010680763 data Select


In [20]:
reco_musics = [_ in filterings[1] for _ in recos_list[1]['id']]
recos_list[1][reco_musics]

,id,name,artists,artists_name
1,0nsL0AalAUFNCi7DYhnihQ,7 Days,1gBUSTR3TyDdTVFIaQnc02,NCT DREAM
2,0mGJTKRIuCOJtiT4YkJHiv,Came Here for Love,1IueXOQyABrMOprrzwQJWN,Sigala
4,37Ctz6MOHM2thzNIOMo7Z0,BOCA,5V1qsQHdXNm4ZEZHWvFnqQ,Dreamcatcher
8,2y3RsLPUpPcMUdzluiJtNE,We DO,5BHFSMEjfLVx1JwRWjAOsE,SuperM
13,2JEaSgRiEra5OQrZlnyqXG,Diggity,1gBUSTR3TyDdTVFIaQnc02,NCT DREAM
19,2hr3geQBSViX9Oi2OsnY3c,Rocket,1gBUSTR3TyDdTVFIaQnc02,NCT DREAM
20,46WaBBaEHzgbN88Ew0nh50,MAGO,0qlWcS66ohOIi0M8JZwPft,GFRIEND
21,3gGyzpBLb7pqOw5oC47QaV,The Purge,2a3wtqb5oDZFegk1xPJu7K,H1GHR MUSIC
22,1FR5w3JYKIgPQ4sf2ARVCn,What I Said,0ziR2zN0NFcB4x1G3P8cW3,VICTON
27,3LcFQsZw01tRvjCNLgSmTv,10 Months,5t5FqBwTcgKTaWmfEbwQY9,ENHYPEN


In [21]:
for idx,mail_box_id in enumerate(mail_boxes_list):
    recos = recos_list[idx]
    filtering = filterings[idx]
    reco_musics = [_ in filtering for _ in recos['id']]
    
    db.new_mail(mail_box_id,
               recos[reco_musics])